In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("patent_scraper/google_list.csv")

In [3]:
df.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-10778703-B2,Method and system for generating an interactiv...,Splunk Inc.,"Sudhakar Muddu, Christos Tryfonas",8/31/15,12/10/18,9/15/20,9/15/20,https://patents.google.com/patent/US10778703B2/en,https://patentimages.storage.googleapis.com/97...
1,US-10762206-B2,Automated behavioral and static analysis using...,"Veracode, Inc.","Theodora H. Titonis, Nelson R. Manohar-Alers, ...",9/16/11,5/16/17,9/1/20,9/1/20,https://patents.google.com/patent/US10762206B2/en,https://patentimages.storage.googleapis.com/f5...
2,CN-107491038-B,"Machine learning machine, numerical control de...",发那科株式会社,"高山堪太, 佐藤和生, 前田英朗",6/9/16,6/2/17,3/3/20,3/3/20,https://patents.google.com/patent/CN107491038B/en,https://patentimages.storage.googleapis.com/30...
3,JP-6898534-B2,Systems and methods to reduce data storage in ...,"トヨタ リサーチ インスティテュート，インコーポレイティド, トヨタ リサーチ インステ...","充啓 馬渕, 充啓 馬渕",3/13/18,2/14/19,7/7/21,7/7/21,https://patents.google.com/patent/JP6898534B2/en,https://patentimages.storage.googleapis.com/f3...
4,US-2020364569-A1,Training reinforcement learning neural networks,Deepmind Technologies Limited,"Hado Philip van Hasselt, Arthur Clément Guez",9/11/15,8/3/20,11/19/20,NaN,https://patents.google.com/patent/US2020036456...,https://patentimages.storage.googleapis.com/13...


In [4]:
df.columns

Index(['id', 'title', 'assignee', 'inventor/author', 'priority date',
       'filing/creation date', 'publication date', 'grant date', 'result link',
       'representative figure link'],
      dtype='object')

In [5]:
len(df['result link'])

19763

In [6]:
df['result link'][0]

'https://patents.google.com/patent/US10778703B2/en'

In [7]:
import os
from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.common.by import By
from selenium import webdriver

In [8]:
driver=webdriver.Chrome()
driver.get(df['result link'][1])

In [9]:
driver.implicitly_wait(1)

In [10]:
pubnum = driver.find_element(By.CSS_SELECTOR, "h2#pubnum")
title = driver.find_element(By.CSS_SELECTOR, "h1#title")
abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")

claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")


print(pubnum.text)
print(title.text)
print(abstract.text)
print("-----")
for claim in claims:
    print(claim.text)

US10762206B2
Automated behavioral and static analysis using an instrumented sandbox and machine learning classification for mobile security
A method comprises, based on receiving a request to analyze at least a first mobile application, scheduling the request for a first sandbox. The first mobile application is analyzed based on the request, wherein the analysis of the first mobile application comprises performing a behavioral analysis of the first mobile application within the first sandbox and performing a static analysis of the first mobile application. A first feature vector is generated based on data resulting from the analysis of the first mobile application. The first mobile application is determined to comprise malware based, at least in part, on comparing the first feature vector with at least a second feature vector, wherein the second feature vector was generated based on at least one of a static analysis and a behavioral analysis of malware.
-----
1. A system comprising:
a 

In [11]:
import json
import os

In [12]:
dataset_folder = "./dataset"
if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)
    print("Dataset folder created")
else:
    print("Dataset folder already exists")

Dataset folder already exists


In [13]:
def create_patent_folder(patent_id):
    patent_folder_path = f"./dataset/{patent_id}"
    if not os.path.exists(patent_folder_path):
        os.makedirs(patent_folder_path)
        print(f"Folder {patent_id} created")
    else:
        print(f"Folder {patent_id} already exists")
    return patent_folder_path

In [46]:
def scrape(patent_link):
    driver=webdriver.Chrome()
    driver.get(patent_link)
    driver.implicitly_wait(1)
    result = {}
    result['patent_link'] = patent_link
    try:
        abstract = driver.find_element(By.CSS_SELECTOR, ".abstract.patent-text")
        result['abstract'] = abstract.text
    except:
        result['abstract'] = ""
    try:
        claims_div = driver.find_element(By.CSS_SELECTOR, ".claims.patent-text")
        claims = claims_div.find_elements(By.CSS_SELECTOR, ".claim-text.patent-text")
        claims_text = ""
        for claim in claims:
            claims_text += claim.text
        result['claims'] = claims_text
    except Exception:
        result['claims'] = ""
    return result

In [47]:
def saveToFile(content):
    file_path = f"./dataset/{content['patent_id']}/{content['patent_id']}.json"
    json_data = json.dumps(content, indent=4)
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
        print(f"File created for {content['patent_id']} at path: {file_path}")

In [48]:
count = 0
for index,row in df.iterrows():
    patent_id = row['id']
    patent_link = row['result link']
    print(f"Fetching patent: {patent_id} from {patent_link}")
    patent_folder_path = create_patent_folder(patent_id)
    content = scrape(patent_link)
    content['patent_id'] = patent_id
    saveToFile(content)
    count += 1
    if count == 10:
        break
print(count)

Fetching patent: US-10778703-B2 from https://patents.google.com/patent/US10778703B2/en
Folder US-10778703-B2 already exists
File created for US-10778703-B2 at path: ./dataset/US-10778703-B2/US-10778703-B2.json
Fetching patent: US-10762206-B2 from https://patents.google.com/patent/US10762206B2/en
Folder US-10762206-B2 already exists
File created for US-10762206-B2 at path: ./dataset/US-10762206-B2/US-10762206-B2.json
Fetching patent: CN-107491038-B from https://patents.google.com/patent/CN107491038B/en
Folder CN-107491038-B already exists
File created for CN-107491038-B at path: ./dataset/CN-107491038-B/CN-107491038-B.json
Fetching patent: JP-6898534-B2 from https://patents.google.com/patent/JP6898534B2/en
Folder JP-6898534-B2 already exists
File created for JP-6898534-B2 at path: ./dataset/JP-6898534-B2/JP-6898534-B2.json
Fetching patent: US-2020364569-A1 from https://patents.google.com/patent/US20200364569A1/en
Folder US-2020364569-A1 already exists
File created for US-2020364569-A1 a